# Vdud Wall

In [1]:
import urllib.request
import json
import pandas as pd
from token_key import k
import ssl
import re
import random
regex = re.compile("[А-ЯЁЙ]{3,}") 

In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

## 0. Функция, читающая url

In [11]:
def readUrl(url, key=k):
    url = url + k
    content = urllib.request.urlopen(url, context=ctx)
    decoded_content = content.read().decode("utf-8")
    data = json.loads(decoded_content)
    return data

## 1. Собираем все id постов со страницы

In [15]:
def post_numbers(owner_id:str, offset_limit:int):
    data = readUrl("https://api.vk.com/method/wall.get?owner_id={}&count=100&v=5.95&access_token=".format(owner_id))
    post_ids = {data["response"]["items"][i]["id"]:data["response"]["items"][i]["text"] for i in range(len(data["response"]["items"]))}
    offset = 101
    while offset < offset_limit:
        data = readUrl("https://api.vk.com/method/wall.get?owner_id={}&offset={}&count=100&v=5.95&access_token=".format(owner_id, offset))
        new_post_ids = {data["response"]["items"][i]["id"]:data["response"]["items"][i]["text"] for i in range(len(data["response"]["items"]))}
        post_ids.update(new_post_ids)
        offset += 100
    return post_ids

In [16]:
post_ids = post_numbers(owner_id="-145800770", 
                        offset_limit=501)

In [17]:
len(post_ids)

368

## 2. Собираем все комментарии с капслоком на русском языке по пост-id

In [33]:
def get_comments(owner_id:str,  
                 post_ids:list,
                offset_limit=1002):

    result_entries = []
    for id_ in post_ids:
        offset = 1
        while offset < offset_limit: 
            data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&post_id={}&offset={}&count=100&v=6.7&access_token=".format(owner_id, str(id_), offset))
            try:
                data = pd.DataFrame(data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
                data_list = data.values.tolist()
                lines = [i[2] for i in data_list]
                offset += 100
                for i, line in enumerate(lines):
                    try:
                        if re.search(regex, line):
                            result_entries.append(data_list[i])
                    except TypeError:
                        continue
                
                comment_ids = [i[3] for i in data_list]
                for com_id in comment_ids:
                    comment_data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&comment_id={}&count=100&v=6.7&access_token=".format(owner_id, com_id))
                    comment_data = pd.DataFrame(comment_data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
                    comment_data_list = comment_data.values.tolist()
                    lines2 = [i[2] for i in comment_data_list]
                    for i, line in enumerate(lines2):
                        try:
                            if re.search(regex, line):
                                result_entries.append(comment_data_list[i])
                        except TypeError:
                            pass
                        
                if offset%100 == 1:
                    print(offset, "ids parsed")

            except KeyError:
                continue
                
    return result_entries

In [18]:
#capslock_comments = get_comments(owner_id="-145800770",
#                                 post_ids=post_ids,
#                                 offset_limit=1002)

In [21]:
offset_limit = 1001
owner_id = "-145800770"

result_entries = []
count = 0
for id_, text in post_ids.items():
    offset = 1
    while offset < offset_limit: 
        data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&post_id={}&offset={}&count=100&v=6.7&access_token=".format(owner_id, id_, offset))
        try:
            data = pd.DataFrame(data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
            data_list = data.values.tolist()
            for i, lst in enumerate(data_list):
                data_list[i].append(text)
            lines = [i[2] for i in data_list]
            offset += 100
            for i, line in enumerate(lines):
                try:
                    if re.search(regex, line):
                        result_entries.append(data_list[i])
                        count += 1
                        if count%100 == 0:
                            print(count, "comments with capslock collected")
                except TypeError:
                    continue
                
            comment_ids_and_text = {i[3]:i[2] for i in data_list}
            for com_id, com_text in comment_ids_and_text.items():
                comment_data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&comment_id={}&count=100&v=6.7&access_token=".format(owner_id, com_id))
                comment_data = pd.DataFrame(comment_data["response"]["items"], columns = ["from_id", "parents_stack", "text", "id"])
                comment_data_list = comment_data.values.tolist()
                for i, lst in enumerate(comment_data_list):
                    comment_data_list[i].append(com_text)
                lines2 = [i[2] for i in comment_data_list]
                for i, line in enumerate(lines2):
                    try:
                        if re.search(regex, line):
                            result_entries.append(comment_data_list[i])
                            count += 1
                            if count%100 == 0:
                                print(count, "comments with capslock collected")
                    except TypeError:
                        pass
        except KeyError:
            continue

100 comments with capslock collected
200 comments with capslock collected
300 comments with capslock collected
400 comments with capslock collected
500 comments with capslock collected
600 comments with capslock collected
700 comments with capslock collected
800 comments with capslock collected
900 comments with capslock collected
1000 comments with capslock collected
1100 comments with capslock collected
1200 comments with capslock collected
1300 comments with capslock collected
1400 comments with capslock collected
1500 comments with capslock collected
1600 comments with capslock collected
1700 comments with capslock collected
1800 comments with capslock collected
1900 comments with capslock collected
2000 comments with capslock collected
2100 comments with capslock collected
2200 comments with capslock collected
2300 comments with capslock collected
2400 comments with capslock collected
2500 comments with capslock collected
2600 comments with capslock collected
2700 comments with ca

In [22]:
df_res = pd.DataFrame(result_entries, columns = ["from_id", "parent_id", "comment_text", "comment_id", "parent_text"])
df_res.count()

from_id         6222
parent_id       6222
comment_text    6222
comment_id      6222
parent_text     6194
dtype: int64

In [23]:
df_res

,from_id,parent_id,comment_text,comment_id,parent_text
0,83439153.0,176744,ААААААААААААА КАКОЕ СЧАСТЬЕ!!!СПАСИБО!!!!,176760,"[club1273675|ГРОТ] - рэп о том, как живет Росс..."
1,32092141.0,176744,Лично знаком с ребятами и с их творчеством оче...,176774,"[club1273675|ГРОТ] - рэп о том, как живет Росс..."
2,339964196.0,176744,"Топовые чуваки, очень крутые ребята. Это не пр...",176854,"[club1273675|ГРОТ] - рэп о том, как живет Росс..."
3,135014878.0,176744,НУ НАКОНЕЦ ТО У РЕАЛНЫХ И АВТОРИТЕТНЫХ!! - ВЗЯ...,176870,"[club1273675|ГРОТ] - рэп о том, как живет Росс..."
4,163060934.0,176744,"Посмотрел на одном дыхании, СПАСИБО!!!👍 Грот -...",176911,"[club1273675|ГРОТ] - рэп о том, как живет Росс..."
5,247971812.0,[176747],"[id59938016|Роман], тут больше подходит, что и...",176885,Вот это действительно крутые гости.
6,247971812.0,[176747],"[id134507320|Максим], а что ты знаешь обо мне,...",177089,Вот это действительно крутые гости.
7,100863978.0,[176747],"[id34739370|Анастасия], вы так распинаетесь, к...",177233,Вот это действительно крутые гости.
8,142553960.0,[176747],"[id247971812|Александр], в текстах ГРОТа больш...",178062,Вот это действительно крутые гости.
9,21276601.0,176744,На конкурс! ...,176988,"[club1273675|ГРОТ] - рэп о том, как живет Росс..."


## 4. Извлекаем инфу о каждом комментаторе и добавляем в датафрейм. 

In [24]:
def chunks(list_of_values, chunk_size=400):
    return [list_of_values[i:i + chunk_size] for i in range(0, len(list_of_values), chunk_size)]

In [25]:
def update_df_with_user_info(df):
    
    df["first_name"] = None 
    df["last_name"] = None 
    df["sex"] = None
    df["bdate"] = None 
    df["city"] = None
    
    df.drop_duplicates(subset=["from_id"], keep='first', inplace=True)
    df["from_id"] = df["from_id"].apply(lambda x: int(x))
    from_ids = [str(df.from_id[i]) for i in df.index]
    df = df.set_index("from_id")
        
    all_data = []
    for ids in chunks(from_ids):
        k = ",".join(ids)
        data = readUrl("https://api.vk.com/method/users.get?user_ids={}&fields=bdate,sex,city&v=6.7&access_token=".format(str(k)))
        all_data += data["response"]
    
    print(all_data[0])
    
    for el in all_data:
        indx = el["id"]
        try:
            el["city"] = el["city"]["title"]
        except KeyError:
            continue
        df.loc[indx, "first_name"] = el.get("first_name", None)
        df.loc[indx, "last_name"] = el.get("last_name", None)
        df.loc[indx, "sex"] = el.get("sex", None)
        df.loc[indx,"bdate"] = el.get("bdate", None)
        df.loc[indx,"city"] = el.get("city", None)
        
    return df

In [26]:
df_res = update_df_with_user_info(df_res)

{'id': 83439153, 'first_name': 'Dmitry', 'last_name': 'Andreev', 'is_closed': True, 'can_access_closed': False, 'sex': 2, 'bdate': '24.11', 'city': {'id': 1, 'title': 'Moscow'}}


In [35]:
df_res.tail()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city
from_id,,,,,,,,,
49407875,22,ДНЕВНИК ХАЧА,247,[id8658426|Амиран Сардаров] (Дневник Хача) - с...,Timofey,Ezhov,2,14.10,Moscow
423900905,22,"Планируется открытие нового канала "" Журнал ЧУ...",2768,[id8658426|Амиран Сардаров] (Дневник Хача) - с...,Svetlana,Mironova,1,6.5.1994,Moscow
1625187,13,"трек ""РУСЗМ"" ( родина умыта слезами зеков и ма...",396,[id2128351|Баста] - о Немагии и своей зарплате...,Sergey,Kostyrev,2,30.1.1987,Blagoveschensk
426915387,13,лох. БАста ЛОХ,1168,[id2128351|Баста] - о Немагии и своей зарплате...,None,None,None,None,None
19670600,13,"Трек должен называться ""Тем кто знает АУЕ"" )",1651,[id2128351|Баста] - о Немагии и своей зарплате...,Vladimir,Kolyaskin,2,31.10,Port-au-Prince


## 5. Удалим комменты с пустыми полями sex, bdate, city

In [36]:
df_test = df_res.dropna(subset=["sex", "bdate", "city"],how="all") 

In [37]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2772 entries, 83439153 to 19670600
Data columns (total 9 columns):
parent_id       2772 non-null object
comment_text    2772 non-null object
comment_id      2772 non-null int64
parent_text     2766 non-null object
first_name      2772 non-null object
last_name       2772 non-null object
sex             2772 non-null object
bdate           2120 non-null object
city            2772 non-null object
dtypes: int64(1), object(8)
memory usage: 129.9+ KB


In [38]:
df_test.sex.value_counts()
# со 100 постов собралось всего 287 комментариев с капслоком 

2    2334
1     438
Name: sex, dtype: int64

In [40]:
df_test.city.value_counts()

Moscow               549
Saint Petersburg     335
Yekaterinburg         62
Novosibirsk           55
Rostov-on-Don         48
Krasnodar             43
Samara                43
Nizhny Novgorod       38
Kazan                 37
Voronezh              36
Minsk                 35
Krasnoyarsk           32
Chelyabinsk           29
Kyiv                  29
Omsk                  28
Ufa                   28
Tomsk                 27
Kaliningrad           26
Perm                  24
Tyumen                23
Volgograd             20
Saratov               19
Penza                 19
Almaty                17
Orenburg              16
Vladivostok           14
Nur-Sultan/Astana     14
Donetsk               13
Yaroslavl             13
Khabarovsk            13
                    ... 
Pavlovskiy Posad       1
Brno                   1
Yakhroma               1
Yaransk                1
Petropavlovsk          1
Salekhard              1
Andorra la Vella       1
Mozdok                 1
Yuzhno-Sakhalinsk      1


In [39]:
df_test.head()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city
from_id,,,,,,,,,
83439153,176744,ААААААААААААА КАКОЕ СЧАСТЬЕ!!!СПАСИБО!!!!,176760,"[club1273675|ГРОТ] - рэп о том, как живет Росс...",Dmitry,Andreev,2,24.11,Moscow
32092141,176744,Лично знаком с ребятами и с их творчеством оче...,176774,"[club1273675|ГРОТ] - рэп о том, как живет Росс...",Andrey,Rodionov,2,14.4,Biysk
163060934,176744,"Посмотрел на одном дыхании, СПАСИБО!!!👍 Грот -...",176911,"[club1273675|ГРОТ] - рэп о том, как живет Росс...",In,Sun,2,16.1,Beloretsk
247971812,[176747],"[id59938016|Роман], тут больше подходит, что и...",176885,Вот это действительно крутые гости.,Alexander,Khvorov,2,15.12.1989,Kotovsk
142553960,[176747],"[id247971812|Александр], в текстах ГРОТа больш...",178062,Вот это действительно крутые гости.,Kirill,Voronkin,2,31.8.2000,Sarov


In [60]:
df_test = df_test.reset_index()

In [65]:
df_test = df_test.drop(columns=["group_info", -1])

In [67]:
df_test = df_test.drop_duplicates(subset=["from_id"])
df_test.count()

from_id         2772
parent_id       2772
comment_text    2772
comment_id      2772
parent_text     2766
first_name      2772
last_name       2772
sex             2772
bdate           2120
city            2772
dtype: int64

## Пишем функцию, которая собирает инфу о группах юзера

In [7]:
df_test = pd.read_csv("hhhhhhhhhhhhhhh.csv", sep="\t")

In [8]:
df_test = df_test.iloc[:,1:]

In [9]:
df_test.head()

,from_id,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city,group_info
0,83439153,176744,ААААААААААААА КАКОЕ СЧАСТЬЕ!!!СПАСИБО!!!!,176760,"[club1273675|ГРОТ] - рэп о том, как живет Росс...",Dmitry,Andreev,2,24.11,Moscow,NaN
1,32092141,176744,Лично знаком с ребятами и с их творчеством оче...,176774,"[club1273675|ГРОТ] - рэп о том, как живет Росс...",Andrey,Rodionov,2,14.4,Biysk,NaN
2,163060934,176744,"Посмотрел на одном дыхании, СПАСИБО!!!👍 Грот -...",176911,"[club1273675|ГРОТ] - рэп о том, как живет Росс...",In,Sun,2,16.1,Beloretsk,NaN
3,247971812,[176747],"[id59938016|Роман], тут больше подходит, что и...",176885,Вот это действительно крутые гости.,Alexander,Khvorov,2,15.12.1989,Kotovsk,NaN
4,142553960,[176747],"[id247971812|Александр], в текстах ГРОТа больш...",178062,Вот это действительно крутые гости.,Kirill,Voronkin,2,31.8.2000,Sarov,NaN


In [43]:
def update_df_with_subskr_info(df, offset_limit=1001):
    
    df = df.set_index("from_id")
    df["group_info"] = None
    indices = list(df.index)
        
    count = 0
    
    for i in indices: 
        offset = 0
        group_info = []
        while offset < offset_limit:
            subskr = readUrl("https://api.vk.com/method/users.getSubscriptions?user_id={}&extended=1&fields=activity,status,description&count=200&offset={}&v=6.7&access_token=".format(str(i), offset))
            offset += 200
            try:
                for j in subskr["response"]["items"]:
                    group = [j["name"], j["status"], j["description"], j["activity"]]
                    group_info.append(group)
            except KeyError:
                continue
            
        if group_info != []:
            group_info = [" -||- ".join(i) for i in group_info]
            group_info = " -|-|- ".join(group_info)
            df.loc[i, "group_info"] = group_info
        
        count += 1
        if count % 10 == 0:
            print(count, "users processed")
        
    return df

In [44]:
df = update_df_with_subskr_info(df_test)

10 users processed
20 users processed
30 users processed
40 users processed
50 users processed
60 users processed
70 users processed
80 users processed
90 users processed
100 users processed
110 users processed
120 users processed
130 users processed
140 users processed
150 users processed
160 users processed
170 users processed
180 users processed
190 users processed
200 users processed
210 users processed
220 users processed
230 users processed
240 users processed
250 users processed
260 users processed
270 users processed
280 users processed
290 users processed
300 users processed
310 users processed
320 users processed
330 users processed
340 users processed
350 users processed
360 users processed
370 users processed
380 users processed
390 users processed
400 users processed
410 users processed
420 users processed
430 users processed
440 users processed
450 users processed
460 users processed
470 users processed
480 users processed
490 users processed
500 users processed
510 users

In [45]:
df.tail()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city,group_info
from_id,,,,,,,,,,
54701393,28,"посмотрел 1/4 выпуска, блевотина!!! В гостях к...",13373,"[id2183360|Соболев] - о Путине, митингах и сек...",Maxim,Demeschenko,2,9.8.1993,Senno,Football House | ТРАНСФЕРЫ -||- Будь в курсе в...
49407875,22,ДНЕВНИК ХАЧА,247,[id8658426|Амиран Сардаров] (Дневник Хача) - с...,Timofey,Ezhov,2,14.10,Moscow,Академия Порядочных Парней -||- 🍌 -||- -||- H...
423900905,22,"Планируется открытие нового канала "" Журнал ЧУ...",2768,[id8658426|Амиран Сардаров] (Дневник Хача) - с...,Svetlana,Mironova,1,6.5.1994,Moscow,СТРЕМНЫЙ ОБЗОР -||- YOUTUBE // МЫ ЕСТЬ БОЛОТО ...
1625187,13,"трек ""РУСЗМ"" ( родина умыта слезами зеков и ма...",396,[id2128351|Баста] - о Немагии и своей зарплате...,Sergey,Kostyrev,2,30.1.1987,Blagoveschensk,NR -||- friday - release day 🎷 -||- Современно...
19670600,13,"Трек должен называться ""Тем кто знает АУЕ"" )",1651,[id2128351|Баста] - о Немагии и своей зарплате...,Vladimir,Kolyaskin,2,31.10,Port-au-Prince,Интересный Екатеринбург -||- Столица Урала -||...


In [46]:
df["source"] = ["vdud" for i in range(len(df))]

In [47]:
df.head()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city,group_info,source
from_id,,,,,,,,,,,
83439153,176744,ААААААААААААА КАКОЕ СЧАСТЬЕ!!!СПАСИБО!!!!,176760,"[club1273675|ГРОТ] - рэп о том, как живет Росс...",Dmitry,Andreev,2,24.11,Moscow,None,vdud
32092141,176744,Лично знаком с ребятами и с их творчеством оче...,176774,"[club1273675|ГРОТ] - рэп о том, как живет Росс...",Andrey,Rodionov,2,14.4,Biysk,None,vdud
163060934,176744,"Посмотрел на одном дыхании, СПАСИБО!!!👍 Грот -...",176911,"[club1273675|ГРОТ] - рэп о том, как живет Росс...",In,Sun,2,16.1,Beloretsk,NR -||- friday - release day 🎷 -||- Современно...,vdud
247971812,[176747],"[id59938016|Роман], тут больше подходит, что и...",176885,Вот это действительно крутые гости.,Alexander,Khvorov,2,15.12.1989,Kotovsk,Мир PlayStation | PS4 -||- playstation.im -||-...,vdud
142553960,[176747],"[id247971812|Александр], в текстах ГРОТа больш...",178062,Вот это действительно крутые гости.,Kirill,Voronkin,2,31.8.2000,Sarov,RapNews / Новости рэпа -||- Самые актуальные н...,vdud


In [48]:
df.reset_index(inplace=True)

In [49]:
df.head()

,from_id,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city,group_info,source
0,83439153,176744,ААААААААААААА КАКОЕ СЧАСТЬЕ!!!СПАСИБО!!!!,176760,"[club1273675|ГРОТ] - рэп о том, как живет Росс...",Dmitry,Andreev,2,24.11,Moscow,None,vdud
1,32092141,176744,Лично знаком с ребятами и с их творчеством оче...,176774,"[club1273675|ГРОТ] - рэп о том, как живет Росс...",Andrey,Rodionov,2,14.4,Biysk,None,vdud
2,163060934,176744,"Посмотрел на одном дыхании, СПАСИБО!!!👍 Грот -...",176911,"[club1273675|ГРОТ] - рэп о том, как живет Росс...",In,Sun,2,16.1,Beloretsk,NR -||- friday - release day 🎷 -||- Современно...,vdud
3,247971812,[176747],"[id59938016|Роман], тут больше подходит, что и...",176885,Вот это действительно крутые гости.,Alexander,Khvorov,2,15.12.1989,Kotovsk,Мир PlayStation | PS4 -||- playstation.im -||-...,vdud
4,142553960,[176747],"[id247971812|Александр], в текстах ГРОТа больш...",178062,Вот это действительно крутые гости.,Kirill,Voronkin,2,31.8.2000,Sarov,RapNews / Новости рэпа -||- Самые актуальные н...,vdud


## 6. Сохраняем все в файл csv

In [50]:
df.to_csv("capitalization-vdud-wall-new.csv", index=False)